In [1]:
# Import Splinter, BeautifulSoup, Regular Expression, and Time
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import time
import wget

In [13]:
executable_path = {'executable_path': 'C:/Users/renie/Desktop/chromedriver.exe'}

browser = Browser('chrome', **executable_path)

In [14]:
months = ["january","march","april","may","june","july","august","september",\
          "october","november","december"]

download_directory = "2019/"
csv_files = []

for month in months:
    url = "https://opendata.dc.gov/datasets/moving-violations-issued-in-" + month + "-2019/data"
    #print(url)
    browser.visit(url)
    time.sleep(2)

    browser.is_element_present_by_text("Spreadsheet", wait_time=1)
    html = browser.html
    violations_soup = bs(html, 'html.parser')
    #violations_soup
    
    link_info = violations_soup.find("a", id="ember104")
    link = link_info['href']
    filename = link_info['download']
    
    csv_files.append(filename)
    print(link)
    print(filename)
#     print()

    download = wget.download(link, out=download_directory)
    print(download)
    
    time.sleep(10)

https://opendata.arcgis.com/datasets/0d7b690c4e874e39a6f006cc61073561_0.csv
moving_violations_issued_in_january_2019.csv
-1 / unknown2019//Moving_Violations_Issued_in_January_2019 (1).csv
https://opendata.arcgis.com/datasets/0e38e123d4414d37905d0bd64af456ad_2.csv
moving_violations_issued_in_march_2019.csv
-1 / unknown2019//Moving_Violations_Issued_in_March_2019 (1).csv
https://opendata.arcgis.com/datasets/878e5e25b4fe47bbbbd3a37c77285a63_3.csv
moving_violations_issued_in_april_2019.csv
-1 / unknown2019//Moving_Violations_Issued_in_April_2019 (1).csv
https://opendata.arcgis.com/datasets/bc4a0785f2f64b979f249b18c4f3fd29_4.csv
moving_violations_issued_in_may_2019.csv
-1 / unknown2019//Moving_Violations_Issued_in_May_2019 (1).csv
https://opendata.arcgis.com/datasets/805a9187829a4840b1094098d5c2c4bb_5.csv
moving_violations_issued_in_june_2019.csv
-1 / unknown2019//Moving_Violations_Issued_in_June_2019 (1).csv
https://opendata.arcgis.com/datasets/6e8387b60cb44adc8a3b917027bc164a_6.csv
moving

In [3]:
column_names = ["ISSUE_DATE","VIOLATION_PROCESS_DESC","DRV_LIC_STATE","FINE_AMOUNT"]

y2019_df = pd.DataFrame(columns = column_names)
print(y2019_df)

first_month = "True"

months = ["January","March","April","May","June","July","August","September",\
          "October","November","December"]

csv_files = []

for month in months:
    filename = "Moving_Violations_Issued_in_" + month + "_2019.csv"
    csv_files.append(filename)

# Make sure the sub-folder name is 2019; otherwise, change it.
for file in csv_files:
    file_path = "2019/" + file
    print(file_path)
    
    month_df = pd.read_csv(file_path)
    
    temp_df = month_df.loc[:,["ISSUE_DATE","VIOLATION_PROCESS_DESC","DRV_LIC_STATE","FINE_AMOUNT"]]
    temp_df = temp_df[temp_df['DRV_LIC_STATE'].isin(["DC","MD","VA"])==True]
    temp_df.dropna(subset=["DRV_LIC_STATE"]) 
    
    if first_month == "True":
        y2019_df = temp_df 
        first_month = "False"
    else:    
        csv_df = pd.concat([y2019_df, temp_df])
        y2019_df = csv_df
        #break
    
y2019_df.to_csv("2019/final.csv")      

Empty DataFrame
Columns: [ISSUE_DATE, VIOLATION_PROCESS_DESC, DRV_LIC_STATE, FINE_AMOUNT]
Index: []
2019/Moving_Violations_Issued_in_January_2019.csv


C:\Users\renie\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2019/Moving_Violations_Issued_in_March_2019.csv
2019/Moving_Violations_Issued_in_April_2019.csv
2019/Moving_Violations_Issued_in_May_2019.csv
2019/Moving_Violations_Issued_in_June_2019.csv
2019/Moving_Violations_Issued_in_July_2019.csv
2019/Moving_Violations_Issued_in_August_2019.csv
2019/Moving_Violations_Issued_in_September_2019.csv
2019/Moving_Violations_Issued_in_October_2019.csv
2019/Moving_Violations_Issued_in_November_2019.csv
2019/Moving_Violations_Issued_in_December_2019.csv


PermissionError: [Errno 13] Permission denied: '2019/final.csv'

In [4]:
y2019_df["YEAR"] = y2019_df['ISSUE_DATE'].str.split('/').str[0]
y2019_df["MONTH"] = y2019_df['ISSUE_DATE'].str.split('/').str[1]

y2019_df.sample(5)

,ISSUE_DATE,VIOLATION_PROCESS_DESC,DRV_LIC_STATE,FINE_AMOUNT,YEAR,MONTH
67090,2019/10/19 04:00:00+00,SPEED 11-15 MPH OVER THE SPEED LIMIT,DC,100.0,2019,10
66936,2019/01/20 05:00:00+00,SPEED 11-15 MPH OVER THE SPEED LIMIT,DC,100.0,2019,01
97164,2019/04/27 04:00:00+00,SPEED 11-15 MPH OVER THE SPEED LIMIT,DC,100.0,2019,04
38079,2019/05/07 04:00:00+00,SPEED 16-20 MPH OVER THE SPEED LIMIT,DC,150.0,2019,05
17081,2019/05/26 04:00:00+00,TURN RIGHT ON RED WITHOUT COMPLETE STOP,DC,100.0,2019,05


### Question 6 - Total amount of issuing violations by State and Month

In [5]:
total_amount = y2019_df.groupby("DRV_LIC_STATE")['FINE_AMOUNT'].sum().map("${:,.1f}".format)

total_amount.to_frame()

,FINE_AMOUNT
DRV_LIC_STATE,
DC,"$25,105,850.0"
MD,"$3,502,470.0"
VA,"$1,287,157.0"


In [6]:
total_amount = y2019_df.groupby(["DRV_LIC_STATE", "MONTH"])['FINE_AMOUNT'].sum().map("${:,.1f}".format)

total_amount.to_frame()

FINE_AMOUNT
DRV_LIC_STATE MONTH              
DC            01     $1,929,450.0
              03     $2,125,015.0
              04     $2,106,375.0
              05     $2,270,000.0
              06     $3,007,080.0
              07     $2,461,670.0
              08     $3,170,890.0
              09     $2,349,595.0
              10     $2,106,245.0
              11     $2,005,436.0
              12     $1,574,094.0
MD            01       $261,086.0
              03       $346,831.0
              04       $302,362.0
              05       $365,972.0
              06       $289,874.0
              07       $303,418.0
              08       $418,166.0
              09       $350,287.0
              10       $310,869.0
              11       $323,488.0
              12       $230,117.0
VA            01       $103,030.0
              03       $150,855.0
              04       $105,652.0
              05       $127,890.0
              06       $110,821.0
              07       $103,686.0
              08       $150,490.0
              09       $131,241.0
              10       $117,500.0
              11       $101,750.0
              12        $84,242.0

In [9]:
violation_amount = y2019_df.groupby("DRV_LIC_STATE")['DRV_LIC_STATE'].count()
violation_amount.to_frame()

,DRV_LIC_STATE
DRV_LIC_STATE,
DC,205096
MD,29238
VA,11152


In [10]:
violation_amount = y2019_df.groupby(["DRV_LIC_STATE", "MONTH"])['DRV_LIC_STATE'].count()
violation_amount.to_frame()

DRV_LIC_STATE
DRV_LIC_STATE MONTH               
DC            01             17305
              03             19181
              04             19012
              05             20333
              06             22917
              07             19144
              08             22851
              09             18417
              10             16425
              11             15773
              12             13738
MD            01              2272
              03              3058
              04              2653
              05              3055
              06              2474
              07              2517
              08              3283
              09              2906
              10              2555
              11              2495
              12              1970
VA            01               916
              03              1361
              04               971
              05              1111
              06               960
              07               879
              08              1183
              09              1142
              10               989
              11               875
              12               765

In [12]:
y2019_df["VIOLATION_PROCESS_DESC"].value_counts()

SPEED 11-15 MPH OVER THE SPEED LIMIT               131347
SPEED 16-20 MPH OVER THE SPEED LIMIT                21434
PASSING STOP SIGN WITHOUT COMING TO A FULL STOP     16890
FAIL TO STOP PER REGULATIONS FACING RED SIGNAL      12516
FAIL TO DISPLAY PROOF OF VEHICLE INSURANCE           5343
                                                    ...  
NO SHIPPING PAPER                                       1
PROVISIONAL PERMIT NIGHT RESTRICTION                    1
DRIVE THROUGH A FUNERAL PROCESSION                      1
VIOLATION OF SNOW EMERGENCY PARKING REGULATIONS         1
NO SNOW TIRE OR CHAIN                                   1
Name: VIOLATION_PROCESS_DESC, Length: 251, dtype: int64